In [1]:
import pandas as pd
import xarray as xr
import numpy as np
from skimage import measure
from shapely.geometry import Polygon
# import matplotlib.pyplot as plt
# from bokeh.models import ColumnDataSource
# from bokeh.plotting import figure, save, show
# from bokeh.palettes import RdYlBu11 as palette
# from bokeh.models import LogColorMapper, Patches

from glob import glob
from os.path import exists, join
from tqdm import tqdm
# import math


In [2]:
path_data = "../data/may/track_data_hrrr_3km_nc_refl/"
path_labels = "../data/may/mod_0_predictions.csv"

input_vars = ["masks", "lat", "lon", "time", "track_id", "track_step"]


In [3]:
def lon_to_web_mercator(lon):
    k = 6378137
    return lon * (k * np.pi / 180.0)

def lat_to_web_mercator(lat):
    k = 6378137
    return np.log(np.tan((90 + lat) * np.pi / 360.0)) * k


In [4]:
def get_xy_coords(storms):
    
    x, y = [],[]
    [(x.append(list(polygon.exterior.coords.xy[0])), y.append(list(polygon.exterior.coords.xy[1]))) for polygon in storms]
    
    return x,y


In [5]:
def get_contours(data, start_idx):
    '''takes storm masks and generates storm outlines in lat-lon coordinates'''
    
    masks = data["masks"].values.astype(np.float32)
    lons = data.variables["lon"].values.astype(np.float32)
    lats = data.variables["lat"].values.astype(np.float32)
    
    storms = []
    skips = []
    for i,mask in enumerate(masks):
        contours = measure.find_contours(mask, 0.7)[0]
        lons_m = []
        lats_m = []
        for contour in np.round(contours).astype(np.int32):
            row = contour[0]
            col = contour[1]
            lons_m.append(lon_to_web_mercator(lons[i][row, col]))
            lats_m.append(lat_to_web_mercator(lats[i][row, col]))
        try:
            storms.append(Polygon(list(zip(lons_m, lats_m))))
        except:
            print(f"Storm {i} doesn't have enough points {list(zip(lons_m, lats_m))} to create Polygon")
            skips.append(start_idx + i)

    x,y = get_xy_coords(storms)
    
    data = data.to_dataframe()
    data = data.reset_index(level=[0,1,2]).drop_duplicates(subset='p', keep='first')
    data = data.set_index('p').drop(['col', 'masks', 'row', 'lat', 'lon'], axis=1)
    data = data.drop(skips)
    data["x"] = x
    data["y"] = y
    
    return data, skips

In [ ]:
# def open_patch_files(path_data):
    
#     filenames = pd.Series(sorted(glob(join(path_data, "*.nc"))))
    
#     dates = filenames.str.split("/").str[-1].str.split("_").str[1]
#     dates = pd.to_datetime(dates)

#     data_list = []
#     for p, patch_file in enumerate(tqdm(filenames[:], ncols=60)):
#         ds = xr.open_dataset(patch_file)
#         ds["run_date"] = xr.DataArray([dates.values[p]] * ds.dims["p"],
#                                          dims=("p",), name="run_date")
#         data_list.append(ds[input_vars+["run_date"]].compute())
#         ds.close()
#     data = xr.concat(data_list, dim="p")
#     data["p"] = np.arange(data["p"].size)
    
#     return data
    

In [9]:
def append_xy_to_labels(labels, path_data, input_vars):

    print("labels.shape", labels.shape)
    labels["x"] = np.nan
    labels["y"] = np.nan

    filenames = pd.Series(sorted(glob(join(path_data, "*.nc"))))
    dates = filenames.str.split("/").str[-1].str.split("_").str[1]
    dates = pd.to_datetime(dates)

    start_idx = 0
    end_idx = 0
    for p, patch_file in enumerate(tqdm(filenames[:], ncols=60)):
        ds = xr.open_dataset(patch_file)
        ds["run_date"] = xr.DataArray([dates.values[p]] * ds.dims["p"],
                                       dims=("p",), name="run_date")
        data = ds[input_vars+["run_date"]].compute()
        ds.close()

        data["p"] = np.arange(start_idx, start_idx + data['p'].shape[0])
        data, skips = get_contours(data, start_idx)
        labels = labels.drop(skips)
        end_idx = start_idx + data.shape[0]
        labels.loc[start_idx:end_idx,'x'] = data['x']
        labels.loc[start_idx:end_idx,'y'] = data['y']
        start_idx = start_idx + data.shape[0]

    print("labels.shape", labels.shape)
    
    return labels
    

In [10]:
labels = pd.read_csv(path_labels)
labels

,time,centroid_lon,centroid_lat,centroid_i,centroid_j,track_id,track_step,run_date,label,cluster,...,cluster_3_prob,cluster_4_prob,cluster_5_prob,cluster_6_prob,S_prob,S,Q_prob,Q,D_prob,D
0,2020-05-01 01:00:00,-105.612015,26.016846,75.0,623.0,0,1,2020-05-01 00:00:00,D,0,...,1.086670e-05,3.141315e-14,5.569079e-10,0.038284,3.773942e-14,0,5.569393e-10,0,1.000000,1
1,2020-05-01 02:00:00,-105.469650,25.763386,65.0,627.0,0,2,2020-05-01 00:00:00,D,0,...,6.904779e-07,2.379865e-15,1.979699e-11,0.009346,1.179140e-15,0,1.979937e-11,0,1.000000,1
2,2020-05-01 01:00:00,-76.874855,26.823439,171.0,1589.0,1,1,2020-05-01 00:00:00,D,6,...,4.384997e-02,2.629617e-10,3.693910e-05,0.797069,6.337175e-09,0,3.693936e-05,0,0.999963,1
3,2020-05-01 02:00:00,-76.519120,27.022020,181.0,1599.0,1,2,2020-05-01 00:00:00,D,3,...,8.283619e-01,3.753821e-06,1.681361e-01,0.003007,4.909102e-04,0,1.681399e-01,0,0.831369,1
4,2020-05-01 03:00:00,-76.291260,27.192207,189.0,1605.0,1,3,2020-05-01 00:00:00,D,0,...,1.858719e-06,9.099239e-15,6.262314e-11,0.018241,3.769444e-15,0,6.263224e-11,0,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255814,2020-06-01 15:00:00,-93.667885,46.325497,822.0,998.0,119,1,2020-05-31 23:00:00,D,3,...,9.219215e-01,5.145431e-07,5.175297e-02,0.026151,5.762594e-05,0,5.175349e-02,0,0.948189,1
255815,2020-06-01 16:00:00,-93.012695,46.278275,821.0,1015.0,119,2,2020-05-31 23:00:00,D,6,...,5.610613e-02,3.201836e-10,5.922789e-05,0.761000,8.263359e-09,0,5.922821e-05,0,0.999941,1
255816,2020-06-01 16:00:00,-98.766624,26.303856,74.0,856.0,120,1,2020-05-31 23:00:00,D,6,...,4.458832e-03,8.234749e-12,8.126207e-07,0.916433,7.841917e-11,0,8.126290e-07,0,0.999999,1
255817,2020-06-01 16:00:00,-95.230720,26.339417,76.0,976.0,121,1,2020-05-31 23:00:00,D,6,...,1.150098e-01,8.245511e-10,1.374111e-04,0.881195,2.930438e-08,0,1.374120e-04,0,0.999863,1


In [11]:
labels = append_xy_to_labels(labels, path_data, input_vars)
labels


  0%|                               | 0/743 [00:00<?, ?it/s]

labels.shape (255819, 23)


 22%|████▋                | 167/743 [01:26<03:24,  2.82it/s]

Storm 55 doesn't have enough points [(-10273110.273217946, 4004890.486413001), (-10273110.273217946, 4004890.486413001)] to create Polygon


 23%|████▊                | 170/743 [01:27<03:30,  2.72it/s]

...

 80%|████████████████▊    | 596/743 [11:02<06:18,  2.57s/it]

Storm 392 doesn't have enough points [(-11026479.569098843, 3174940.0659196954), (-11023101.901748953, 3171673.0298292963)] to create Polygon


100%|█████████████████████| 743/743 [15:01<00:00,  1.21s/it]

labels.shape (255814, 25)


,time,centroid_lon,centroid_lat,centroid_i,centroid_j,track_id,track_step,run_date,label,cluster,...,cluster_5_prob,cluster_6_prob,S_prob,S,Q_prob,Q,D_prob,D,x,y
0,2020-05-01 01:00:00,-105.612015,26.016846,75.0,623.0,0,1,2020-05-01 00:00:00,D,0,...,5.569079e-10,0.038284,3.773942e-14,0,5.569393e-10,0,1.000000,1,"[-11768770.591680275, -11768770.591680275, -11...","[3026367.66760143, 3026367.66760143, 3022813.1..."
1,2020-05-01 02:00:00,-105.469650,25.763386,65.0,627.0,0,2,2020-05-01 00:00:00,D,0,...,1.979699e-11,0.009346,1.179140e-15,0,1.979937e-11,0,1.000000,1,"[-11756675.705904618, -11756675.705904618, -11...","[3001167.5055668745, 3001167.5055668745, 29979..."
2,2020-05-01 01:00:00,-76.874855,26.823439,171.0,1589.0,1,1,2020-05-01 00:00:00,D,6,...,3.693910e-05,0.797069,6.337175e-09,0,3.693936e-05,0,0.999963,1,"[-8547554.551283864, -8547554.551283864, -8551...","[3116055.0122915804, 3116055.0122915804, 31135..."
3,2020-05-01 02:00:00,-76.519120,27.022020,181.0,1599.0,1,2,2020-05-01 00:00:00,D,3,...,1.681361e-01,0.003007,4.909102e-04,0,1.681399e-01,0,0.831369,1,"[-8517322.007996447, -8520545.10268914, -85237...","[3129445.9593873946, 3130193.361561164, 313093..."
4,2020-05-01 03:00:00,-76.291260,27.192207,189.0,1605.0,1,3,2020-05-01 00:00:00,D,0,...,6.262314e-11,0.018241,3.769444e-15,0,6.263224e-11,0,1.000000,1,"[-8480163.42064608, -8480163.42064608, -848415...","[3171829.177637232, 3171829.177637232, 3169357..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255814,2020-06-01 15:00:00,-93.667885,46.325497,822.0,998.0,119,1,2020-05-31 23:00:00,D,3,...,5.175297e-02,0.026151,5.762594e-05,0,5.175349e-02,0,0.948189,1,NaN,NaN
255815,2020-06-01 16:00:00,-93.012695,46.278275,821.0,1015.0,119,2,2020-05-31 23:00:00,D,6,...,5.922789e-05,0.761000,8.263359e-09,0,5.922821e-05,0,0.999941,1,NaN,NaN
255816,2020-06-01 16:00:00,-98.766624,26.303856,74.0,856.0,120,1,2020-05-31 23:00:00,D,6,...,8.126207e-07,0.916433,7.841917e-11,0,8.126290e-07,0,0.999999,1,NaN,NaN
255817,2020-06-01 16:00:00,-95.230720,26.339417,76.0,976.0,121,1,2020-05-31 23:00:00,D,6,...,1.374111e-04,0.881195,2.930438e-08,0,1.374120e-04,0,0.999863,1,NaN,NaN


In [12]:
labels.to_csv("../data/may/mod_0_predictions_wxy.csv")

In [15]:
!ls ../data/may/*.csv

../data/may/mod_0_predictions.csv    ../data/may/mod_0_predictions_xy.csv


In [16]:
run_date = '2020-05-01 00:00:00'


In [22]:
labels.loc[labels['run_date'] == run_date, "time"].min().to_numpy()

AttributeError: 'str' object has no attribute 'to_numpy'